In [37]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [38]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')

In [39]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['week', 'away_team', 'away_score', 'home_team', 'home_score']]


In [40]:
# Get data from games
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.0,0.031250,-0.012397,0.038197,0.0,0.80,1.80
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.2,0.035714,0.089286,0.116071,0.0,1.60,2.60
3,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.46,14.46
4,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,2,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.98,13.98


In [41]:
#clean up data to show data columns for each game
columns = [ 'week','recent_team','opponent_team','passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa'
           ]


In [42]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,recent_team,opponent_team,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa
0,1,NYJ,BUF,-2.031960,0.0,NaN,0.0,NaN
1,4,CHI,DEN,NaN,0.0,NaN,0.0,0.483465
2,7,CHI,LV,NaN,0.0,NaN,0.0,1.437224
3,1,LA,SEA,20.679981,334.0,0.868086,11.0,NaN
4,2,LA,SF,-5.089193,307.0,-0.438330,17.0,NaN


In [89]:
# Groupby team and week offense stats
team_by_week = games_df.groupby(['week', 'recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_offense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off = team_stats_per_game.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off = team_stats_per_game_off.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off.tail()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
207,7,PIT,0.638551,230.0,0.346571,86.0,7.945315
208,7,SEA,7.281663,219.0,-7.570456,115.0,11.220156
209,7,SF,8.158350,272.0,-4.455659,65.0,10.399721
210,7,TB,0.575186,275.0,-3.841442,73.0,2.110172
211,7,WAS,-14.697247,249.0,-0.724210,76.0,0.793286


In [44]:
# Groupby team and week defense stats
team_by_week_def = games_df.groupby(['week', 'opponent_team'])
team_by_week_def.head()
team_stats_per_game_def = team_by_week_def.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def = team_stats_per_game_def.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game_def = team_stats_per_game_def.reset_index()


In [45]:
team_stats_per_game_def.head()

,week,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
1,1,ATL,-10.528563,146.0,-8.897445,154.0,-3.103000
2,1,BAL,-11.156192,242.0,-8.125071,72.0,2.796677
3,1,BUF,-5.221493,140.0,2.803440,172.0,3.475710
4,1,CAR,-4.450450,115.0,-0.595772,130.0,0.718799


In [46]:
# merge both dfs so it shows offense and defense stats per game
team_total_df = team_stats_per_game.merge(team_stats_per_game_def, left_on='team_offense', right_on='team_defense')
# Filter rows where 'team_offense' is equal to 'team_defense'
team_total_df = team_total_df[team_total_df['team_offense'] == team_total_df['team_defense']]
#remove duplicates
team_total_df = team_total_df[(team_total_df['team_offense'] == team_total_df['team_defense']) & (team_total_df['week_x'] == team_total_df['week_y'])]
team_total_df.head(1)

,week_x,team_offense,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa,week_y,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.79934,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773


In [47]:
# create a copy of team_total_df for home team and away team

# Make the first copy of the original DataFrame
home_team_df = team_total_df.copy()

# Make a second copy from the first copy
away_team_df = home_team_df.copy()

In [48]:
#make a home_team df
# Add '_home' to each column name
for column in home_team_df.columns:
    new_column_name_home = column + '_home'
    home_team_df.rename(columns={column: new_column_name_home}, inplace=True)
    
home_team_df.head()

,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
8,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
16,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
24,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
32,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [49]:
#make a away_team df
# Add '_away' to each column name
for column in away_team_df.columns:
    new_column_name_away = column + '_away'
    away_team_df.rename(columns={column: new_column_name_away}, inplace=True)
away_team_df.head()

,week_x_away,team_offense_away,passing_epa_away,passing_yards_away,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
8,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
16,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
24,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
32,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [88]:
# Make df with points and stats

# merge home team with home stats
home_team = team_vs_team_df.merge(home_team_df, left_on='home_team', right_on='team_offense_home')
#remove duplicates
home_team = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
home_team.tail(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
1581,4,ARI,16.0,SF,35.0,4,SF,22.844091,283.0,2.675849,124.0,24.983665,4,SF,3.898311,265.0,5.478536,105.0,6.421503
1589,5,DAL,10.0,SF,42.0,5,SF,17.486433,253.0,-6.083300,170.0,18.508959,5,SF,-13.610493,161.0,-9.731944,57.0,-7.575013
1629,3,NO,17.0,GB,18.0,3,GB,-2.964409,259.0,1.365948,95.0,0.069139,3,GB,-2.445023,204.0,-0.864812,77.0,4.414512
1636,4,DET,34.0,GB,20.0,4,GB,-12.693180,246.0,0.423209,27.0,-5.993364,4,GB,2.459123,210.0,4.333835,211.0,6.932714
1677,3,DEN,20.0,MIA,70.0,3,MIA,31.062447,376.0,12.792768,350.0,31.062447,3,MIA,2.208719,306.0,-5.999615,69.0,-5.733638
1686,5,NYG,16.0,MIA,31.0,5,MIA,-1.844198,308.0,7.101428,222.0,-0.270984,5,MIA,-2.154711,205.0,-9.014591,85.0,9.488153
1694,6,CAR,21.0,MIA,42.0,6,MIA,7.147842,262.0,2.244307,162.0,9.160931,6,MIA,-12.907183,224.0,3.199303,108.0,-3.217727
1740,3,PIT,23.0,LV,18.0,3,LV,-8.352474,324.0,-2.516379,69.0,-2.013242,3,LV,5.921596,235.0,-6.737266,105.0,9.555780
1749,5,GB,13.0,LV,17.0,5,LV,-1.803250,208.0,-3.381476,96.0,6.604667,5,LV,-14.767113,182.0,4.087527,110.0,-11.481108
1757,6,NE,17.0,LV,21.0,6,LV,9.882565,264.0,-10.972410,84.0,11.559125,6,LV,-4.471819,200.0,4.164759,83.0,3.130720


In [51]:
# merge away team with home stats
away_team = home_team.merge(away_team_df, left_on='away_team', right_on='team_offense_away')
#remove duplicates
away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
away_team.head(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,...,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,DET,21.0,KC,20.0,1,KC,-6.453613,226.0,-2.581925,...,-3.636585,118.0,6.054341,1,DET,-6.453613,226.0,-2.581925,90.0,-5.052831
13,7,DET,6.0,BAL,38.0,7,BAL,24.334782,357.0,1.219639,...,1.834399,84.0,0.542692,7,DET,24.334782,357.0,1.219639,146.0,24.334782
19,6,DET,20.0,TB,6.0,6,TB,-8.300112,206.0,-4.832714,...,-7.840065,40.0,22.679295,6,DET,-8.300112,206.0,-4.832714,46.0,-7.366295
24,4,DET,34.0,GB,20.0,4,GB,-12.693180,246.0,0.423209,...,4.333835,211.0,6.932714,4,DET,-12.693180,246.0,0.423209,27.0,-5.993364
30,3,CHI,10.0,KC,41.0,3,KC,14.211285,303.0,0.524553,...,2.051218,116.0,-9.638733,3,CHI,14.211285,303.0,0.524553,153.0,14.669461
39,5,CHI,40.0,WAS,20.0,5,WAS,3.675323,388.0,-5.300707,...,8.263792,178.0,17.594566,5,CHI,3.675323,388.0,-5.300707,29.0,5.126959
43,2,CHI,17.0,TB,27.0,2,TB,13.258485,317.0,-1.234122,...,0.217876,67.0,-1.142673,2,CHI,13.258485,317.0,-1.234122,120.0,14.781831
54,6,DEN,8.0,KC,19.0,6,KC,4.125069,306.0,-9.408838,...,4.712027,115.0,-16.197714,6,DEN,4.125069,306.0,-9.408838,96.0,9.299211
59,4,DEN,31.0,CHI,28.0,4,CHI,7.142285,335.0,-0.285303,...,2.192693,97.0,16.564686,4,DEN,7.142285,335.0,-0.285303,171.0,21.835260
65,3,DEN,20.0,MIA,70.0,3,MIA,31.062447,376.0,12.792768,...,-5.999615,69.0,-5.733638,3,DEN,31.062447,376.0,12.792768,350.0,31.062447


In [52]:
# rename df
full_df = away_team.copy()

In [94]:
full_df.to_excel('training_nfl.xlsx', index=False)


In [54]:
# Create Columns for training
column_train_x = [
   'passing_epa_home','passing_yards_home','rushing_epa_home','rushing_yards_home','receiving_epa_home',
    'passing_epa_def_home','passing_yards_def_home','rushing_epa_def_home','rushing_yards_def_home','receiving_epa_def_home',
       'passing_epa_away','passing_yards_away','rushing_epa_away','rushing_yards_away','receiving_epa_away',
           'passing_epa_def_away','passing_yards_def_away','rushing_epa_def_away','rushing_yards_def_away','receiving_epa_def_away',


]




column_train_y = ['home_score','away_score']

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = full_df[column_train_x]
y = full_df[column_train_y]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 4.60887746478236
Mean Squared Error (MSE): 36.049393246244314
Root Mean Squared Error (RMSE): 5.993574613211465
R-squared (R2): 0.6601143591192067


In [56]:
team_stats_per_game_off.head()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340
1,1,ATL,-4.450450,115.0,-0.595772,130.0,0.718799
2,1,BAL,-4.415663,169.0,0.155152,110.0,7.985488
3,1,BUF,-1.409397,236.0,-3.168568,97.0,5.100840
4,1,CAR,-10.528563,146.0,-8.897445,154.0,-3.103000


In [57]:
### Create mean of these stats to create test data
#home team
mean_offense = team_stats_per_game_off.groupby('team_offense').mean()
mean_offense = mean_offense.drop('week', axis=1)
mean_offense = mean_offense.reset_index()

mean_offense.head()

,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,-5.828878,195.0,0.274228,139.000000,0.116334
1,ATL,-2.940421,236.0,-4.291197,124.285713,2.051687
2,BAL,3.507788,230.0,-0.286044,145.000000,8.000455
3,BUF,9.134677,263.0,2.263680,112.857140,10.827784
4,CAR,-7.374483,222.5,-2.244786,98.000000,2.970957


In [58]:
#defense team
# Split the DataFrame into non-numerical and numerical columns
mean_defense = team_stats_per_game_def.groupby('team_defense').mean()
# Remove the week column
mean_defense = mean_defense.drop('week', axis=1)
mean_defense = mean_defense.reset_index()

mean_defense.head()


,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,ARI,4.857561,259.571442,-0.077122,130.714279,10.759151
1,ATL,0.413798,203.142853,-4.970017,95.285713,4.122190
2,BAL,-9.181587,210.000000,-3.010455,95.714287,-0.075023
3,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503
4,CAR,0.209295,214.000000,3.305121,144.333328,4.945242


In [59]:
# Merge on defense vs offense per team
test_data_df = pd.merge(mean_defense, mean_offense, left_on='team_defense',right_on='team_offense', how='left')
test_data_df.head()

,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,4.857561,259.571442,-0.077122,130.714279,10.759151,ARI,-5.828878,195.0,0.274228,139.000000,0.116334
1,ATL,0.413798,203.142853,-4.970017,95.285713,4.122190,ATL,-2.940421,236.0,-4.291197,124.285713,2.051687
2,BAL,-9.181587,210.000000,-3.010455,95.714287,-0.075023,BAL,3.507788,230.0,-0.286044,145.000000,8.000455
3,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503,BUF,9.134677,263.0,2.263680,112.857140,10.827784
4,CAR,0.209295,214.000000,3.305121,144.333328,4.945242,CAR,-7.374483,222.5,-2.244786,98.000000,2.970957


In [60]:
# create DF for upcoming week
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(8)
get_teams =['home_team','away_team','home_score','away_score']
schedule_week_df = week_6_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team,home_score,away_score
0,BUF,TB,NaN,NaN
1,CAR,HOU,NaN,NaN
2,DAL,LA,NaN,NaN
3,GB,MIN,NaN,NaN
4,IND,NO,NaN,NaN


In [61]:
# create a copy of team_total_df for home team and away team for testing

# Make the first copy of the original DataFrame
home_team_test = test_data_df.copy()

# Make a second copy from the first copy
away_team_test = home_team_test.copy()

In [62]:
# Add '_home' to each column name
for column in home_team_test.columns:
    new_column_name_test = column + '_home'
    home_team_test.rename(columns={column: new_column_name_test}, inplace=True)
    
home_team_test.head()

,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ARI,4.857561,259.571442,-0.077122,130.714279,10.759151,ARI,-5.828878,195.0,0.274228,139.000000,0.116334
1,ATL,0.413798,203.142853,-4.970017,95.285713,4.122190,ATL,-2.940421,236.0,-4.291197,124.285713,2.051687
2,BAL,-9.181587,210.000000,-3.010455,95.714287,-0.075023,BAL,3.507788,230.0,-0.286044,145.000000,8.000455
3,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503,BUF,9.134677,263.0,2.263680,112.857140,10.827784
4,CAR,0.209295,214.000000,3.305121,144.333328,4.945242,CAR,-7.374483,222.5,-2.244786,98.000000,2.970957


In [63]:
# Add '_away' to each column name
for column in away_team_test.columns:
    new_column_name_test_away = column + '_away'
    away_team_test.rename(columns={column: new_column_name_test_away}, inplace=True)
    
away_team_test.head()

,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ARI,4.857561,259.571442,-0.077122,130.714279,10.759151,ARI,-5.828878,195.0,0.274228,139.000000,0.116334
1,ATL,0.413798,203.142853,-4.970017,95.285713,4.122190,ATL,-2.940421,236.0,-4.291197,124.285713,2.051687
2,BAL,-9.181587,210.000000,-3.010455,95.714287,-0.075023,BAL,3.507788,230.0,-0.286044,145.000000,8.000455
3,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503,BUF,9.134677,263.0,2.263680,112.857140,10.827784
4,CAR,0.209295,214.000000,3.305121,144.333328,4.945242,CAR,-7.374483,222.5,-2.244786,98.000000,2.970957


In [64]:
# Make df with points and stats for testing upcoming week

# merge home team with home stats
testing_home = schedule_week_df.merge(home_team_test, left_on='home_team', right_on='team_offense_home')
#remove duplicates
#testing_home = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
testing_home.head(1)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,BUF,TB,NaN,NaN,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503,BUF,9.134677,263.0,2.26368,112.85714,10.827784


In [65]:
# merge away team with home stats
testing_away = testing_home.merge(away_team_test, left_on='away_team', right_on='team_offense_away')
#remove duplicates
#away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
testing_away.head(10)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,...,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,BUF,TB,NaN,NaN,BUF,-2.528141,223.428574,-1.562072,128.285721,3.655503,...,262.500000,-4.504119,95.833336,6.143928,TB,2.523806,227.166672,-5.618269,77.833336,4.473943
1,CAR,HOU,NaN,NaN,CAR,0.209295,214.000000,3.305121,144.333328,4.945242,...,254.000000,-2.376219,108.500000,5.636856,HOU,4.192589,277.666656,-4.997757,88.833336,11.226711
2,DAL,LA,NaN,NaN,DAL,-5.953949,192.666672,-4.343773,111.666664,-1.763129,...,220.714279,-0.236637,117.000000,3.619319,LA,2.243471,272.571442,1.824937,112.000000,7.743327
3,GB,MIN,NaN,NaN,GB,-0.928787,211.500000,0.387513,143.666672,4.968079,...,245.428574,-2.851722,105.571426,8.572520,MIN,2.514730,293.857147,-1.508042,74.857140,5.441100
4,IND,NO,NaN,NaN,IND,-0.228434,253.428574,-2.844873,118.714287,6.021268,...,200.571426,-3.026583,98.714287,-1.984794,NO,-3.482750,246.571426,-3.289721,98.142860,2.280386
5,MIA,NE,NaN,NaN,MIA,1.128432,256.285706,0.502756,112.285713,5.701513,...,224.142853,-3.999933,100.714287,8.185001,NE,-5.273420,222.714279,-3.880849,85.428574,-3.168858
6,NYG,NYJ,NaN,NaN,NYG,0.400337,225.714279,1.517380,137.285721,3.548513,...,228.500000,-2.494067,135.166672,4.372197,NYJ,-8.733444,182.833328,-1.755343,117.500000,-2.042526
7,PIT,JAX,NaN,NaN,PIT,-3.818531,259.666656,-0.028367,142.333328,3.301476,...,284.000000,-2.159520,80.571426,-0.050798,JAX,-0.569323,237.714279,-3.994484,113.428574,4.182541
8,TEN,ATL,NaN,NaN,TEN,5.338070,256.666656,-5.067122,102.000000,10.639641,...,203.142853,-4.970017,95.285713,4.122190,ATL,-2.940421,236.000000,-4.291197,124.285713,2.051687
9,WAS,PHI,NaN,NaN,WAS,2.579691,263.571442,-0.125513,125.714287,9.469135,...,248.571426,-1.801599,62.857143,3.923811,PHI,1.718136,260.142853,2.534633,142.714279,6.187350


In [66]:
# rename df
full_test = testing_away.copy()
#full_test = full_test.reset_index()

full_test = full_test.rename(columns={
    'passing_epa_offense_away': 'passing_epa_away',
    'passing_yards_offense_away': 'passing_yards_away',
    'receiving_epa_offense_away': 'receiving_epa_away',
    'rushing_epa_offense_away': 'rushing_epa_away',
    'rushing_yards_offense_away': 'rushing_yards_away',
     'passing_epa_offense_home': 'passing_epa_home',
    'passing_yards_offense_home': 'passing_yards_home',
    'receiving_epa_offense_home': 'receiving_epa_home',
    'rushing_epa_offense_home': 'rushing_epa_home',
    'rushing_yards_offense_home': 'rushing_yards_home',
})
full_test.dtypes

home_team                  object
away_team                  object
home_score                float64
away_score                float64
team_defense_home          object
passing_epa_def_home      float32
passing_yards_def_home    float32
rushing_epa_def_home      float32
rushing_yards_def_home    float32
receiving_epa_def_home    float32
team_offense_home          object
passing_epa_home          float32
passing_yards_home        float32
rushing_epa_home          float32
rushing_yards_home        float32
receiving_epa_home        float32
team_defense_away          object
passing_epa_def_away      float32
passing_yards_def_away    float32
rushing_epa_def_away      float32
rushing_yards_def_away    float32
receiving_epa_def_away    float32
team_offense_away          object
passing_epa_away          float32
passing_yards_away        float32
rushing_epa_away          float32
rushing_yards_away        float32
receiving_epa_away        float32
dtype: object

In [67]:
column_test_x = ['passing_epa_def_away', 'passing_yards_def_away',
       'rushing_epa_def_away', 'rushing_yards_def_away',
       'receiving_epa_def_away','passing_epa_away', 'passing_yards_away',
    'rushing_epa_away', 'rushing_yards_away','receiving_epa_away',
     'passing_epa_def_home', 'passing_yards_def_home',
       'rushing_epa_def_home', 'rushing_yards_def_home',
       'receiving_epa_def_home','passing_epa_home', 'passing_yards_home',
    'rushing_epa_home', 'rushing_yards_home',
       'receiving_epa_home',            
                ]

In [68]:
# Now, use the trained model to predict score for df_test
X_test = full_test[column_test_x]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
print(y_pred)

Predicted Scores:
[[24.407745  17.086594 ]
 [20.307323  22.733791 ]
 [22.081882  18.675209 ]
 [21.23677   19.360332 ]
 [20.644714  19.361359 ]
 [27.105333  17.70575  ]
 [19.773796  19.9967   ]
 [19.791042  20.247244 ]
 [19.434792  18.041498 ]
 [20.18148   23.183226 ]
 [20.719376  14.9334755]
 [17.861637  26.003525 ]
 [19.590286  26.350838 ]
 [25.792095  14.334122 ]
 [24.645348  21.075531 ]
 [24.290886  15.7543545]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [69]:
# add predicted scores into df: 
predict_cols = ['home_team','away_team']
df_predictions = full_test[predict_cols]
scores_df = pd.DataFrame(y_pred, columns=['home_score', 'away_score'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)
df_predictions['Over/Under'] = df_predictions['home_score'] + df_predictions['away_score']
# Assuming 'home_score', 'away_score', 'home_team', and 'away_team' are columns in your DataFrame
df_predictions['Favorite'] = np.where(df_predictions['home_score'] > df_predictions['away_score'], df_predictions['home_team'], df_predictions['away_team'])
df_predictions['Spread'] = df_predictions['away_score'] - df_predictions['home_score']
df_predictions['spread'] = df_predictions['Spread'].apply(lambda x: '-' + str(x) if x >= 0 else str(x))
df_predictions = df_predictions.drop('Spread', axis=1)

df_predictions.head(35)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,BUF,TB,24.407745,17.086594,41.494339,BUF,-7.3211517333984375
1,CAR,HOU,20.307323,22.733791,43.041115,HOU,-2.4264678955078125
2,DAL,LA,22.081882,18.675209,40.757092,DAL,-3.4066734313964844
3,GB,MIN,21.236771,19.360332,40.597103,GB,-1.8764381408691406
4,IND,NO,20.644714,19.361359,40.006073,IND,-1.283355712890625
5,MIA,NE,27.105333,17.705750,44.811081,MIA,-9.39958381652832
6,NYG,NYJ,19.773796,19.996700,39.770496,NYJ,-0.22290420532226562
7,PIT,JAX,19.791042,20.247244,40.038284,JAX,-0.45620155334472656
8,TEN,ATL,19.434792,18.041498,37.476288,TEN,-1.3932933807373047
9,WAS,PHI,20.181480,23.183226,43.364708,PHI,-3.0017452239990234


In [70]:
df_predictions.to_excel('week_8_pred.xlsx', index=False)


In [73]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()










 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2023 13:50:27] "GET / HTTP/1.1" 200 -
